In [ ]:
import chess
import numpy as np
from chess import engine
from tensorflow.keras import models

In [ ]:
def bitboard(board: chess.Board) -> np.ndarray:
    bitmap = np.zeros(
        (len(chess.FILE_NAMES), len(chess.RANK_NAMES), len(chess.PIECE_TYPES)), dtype=int
    )
    for piece_idx, piece_type in enumerate(chess.PIECE_TYPES):
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece and piece.piece_type is piece_type:
                bitmap[
                    chess.square_rank(square),
                    chess.square_file(square),
                    piece_idx
                ] = 1 if piece.color else -1
    return bitmap


def absolute_bitboard(board: chess.Board) -> np.ndarray:
    bitmap = bitboard(board)
    return np.dstack((bitmap, np.full(bitmap.shape[:-1], 1 if board.turn else -1)))


def relative_bitboard(board: chess.Board) -> np.ndarray:
    bitmap = bitboard(board)
    return bitmap if board.turn else np.flip(bitmap, axis=range(bitmap.ndim - 1)) * -1

In [ ]:
board = chess.Board(chess.STARTING_FEN)

In [ ]:
model = models.load_model("ser/v2")

In [ ]:
simple_engine = engine.SimpleEngine.popen_uci("../lib/stockfish/stockfish")

In [ ]:
(
    model.predict(np.array([relative_bitboard(board)])),
    simple_engine.analyse(board, engine.Limit(time=.1))["score"]
)

In [ ]:
move = board.push_san("d4")
# board.pop()
board